In [20]:
# Toy example given by gurobi

from gurobipy import *

try:
    # create a new model
    m = Model("mip1")

    # create variables
    x = m.addVar(vtype=GRB.BINARY, name="x")
    y = m.addVar(vtype=GRB.BINARY, name="y")
    z = m.addVar(vtype=GRB.BINARY, name="z")

    # Establish objective

    m.setObjective(x+y+2*z, GRB.MAXIMIZE)

    # Add constraint 1
    m.addConstr(x+2*y+3*z<=4,"c0")
    # Add constraint 2
    m.addConstr(x+y>=1,"c1")
    m.optimize()
    for v in m.getVars():
        print(v.varName,v.x)
    print('Obj:', m.objVal)
except GurobiError:
    print('Error reported')

Optimize a model with 2 rows, 3 columns and 5 nonzeros
Variable types: 0 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 2 rows and 3 columns
Presolve time: 0.10s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.28 seconds
Thread count was 1 (of 4 available processors)

Solution count 2: 3 2 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%
x 1.0
y 0.0
z 1.0
Obj: 3.0


In [14]:
import pandas as pd
import numpy as np
from gurobipy import *

################################################################################
############################### New formulation ################################
################################################################################

### The second formulation works by constraining the max class sizes, 
### rather than constrain them. This is to analyze the different effects of 
### formulating similar, but different problems


### Read in the data (first choice, second choice, third choice)

first_choice = pd.read_csv("../Resources/FirstChoiceBinary.csv")
second_choice = pd.read_csv("../Resources/SecondChoiceBinary.csv")
third_choice = pd.read_csv("../Resources/ThirdChoiceBinary.csv")


# Make COURSES dictionary
COURSES = {}
i = 1
for course in first_choice.columns[1:]:
    COURSES[i] = course
    i += 1
    
# Make STUDENTS dictionary
STUDENTS = {}
i = 1
for student in first_choice.iloc[:,0]:
    STUDENTS[i] = student
    i += 1

Schedule1 = np.zeros([first_choice.shape[0], first_choice.shape[1]-1])
for i in range(first_choice.shape[0]):
    Schedule1[i] = first_choice.iloc[i,1:].tolist()
    
Schedule2 = np.zeros([second_choice.shape[0], second_choice.shape[1]-1])
for i in range(second_choice.shape[0]):
    Schedule2[i] = second_choice.iloc[i,1:].tolist()
    
Schedule3 = np.zeros([third_choice.shape[0], third_choice.shape[1]-1])
for i in range(third_choice.shape[0]):
    Schedule3[i] = third_choice.iloc[i,1:].tolist()



In [11]:
# Verify everything is the same:
if (len(first_choice.columns) == len(second_choice.columns)) and len(second_choice.columns) == len(third_choice.columns):
    print("Same number of columns")
    
### course size information
course_sizes = pd.read_csv("Resources/CourseSize.csv")

Same number of columns


In [28]:
# construct the model 

m = Model("second formulation")

### binary variables for whether a student gets a schedule
X = {} # variable dictioary
for i in range(1, len(STUDENTS)+1):
    
    for j in [1,2,3]:
        
        name = str(STUDENTS[i]) + " pref" + str(j)
        
        # why is this being assigned here
        
        X[i,j] = m.addVar(name=name, vtype=GRB.BINARY)

        
### add the max_course variable 

max_courses = m.addVar(vtype=GRB.INTEGER,name="max_courses")


### objective is the max class size

m.setObjective(max_courses, GRB.MINIMIZE)


# Assignment Constraint

for s in range(len(STUDENTS)):
    
    m.addConstr(X[s+1,1] + X[s+1,2] + X[s+1,3] == 1)
    
    
### Constraint on the number of 1st choices vs 2nd choices vs 3rd choices
m.addConstr(quicksum(X[s,1] for s in STUDENTS) - quicksum(X[s,2] for s in STUDENTS) > 0)
m.addConstr(quicksum(X[s,2] for s in STUDENTS) - quicksum(X[s,3] for s in STUDENTS) > 0)

# max_course must be greater than the actual assigned number of students Constraint

for c in range(len(COURSES)):
    m.addConstr(quicksum(X[s,1]*Schedule1[s-1,c] + X[s,2]*Schedule2[s-1,c] + X[s,3]*Schedule3[s-1,c]
                       for s in range(1, len(STUDENTS)+1)) <= max_courses)
    

# optimize
m.optimize()

NotImplementedError: 